In [ ]:
#Python Packages
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb 
import csv
import itertools
from pathlib import Path
from pandas import *
import os.path
import matplotlib.pyplot as plt
import cv2

In [ ]:
#Load in functions from freelymoving ephys
import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/FreelyMovingEphys')
from utils.paths import find
from utils.time import open_time1
from utils.time import open_time
from utils.format_data import open_h5
#from utils.format_data import open_ma_h5

#function to get correct frame in case there's lag in the acquisition
def find_first(item, vec):
    return np.argmin(np.abs(vec-item))

In [ ]:
##Load Data
# Dictionary of columns in data frame
data_dict = {'date': [],
             'animal': [],
             'task': [],             
             'poke1_ts':[],
             'poke2_ts': [],
             'spout1_ts': [],
             'spout2_ts': [],
             'top1_ts': [],
             'poke1_t0':[],
             'poke2_t0': [],
             'spout1_t0': [],
             'spout2_t0': [],
             'top1_t0': []}
# List of dates for analysis
dates= ['090221']
data_path = Path('~/Desktop/mike_bonsai/recordings/').expanduser()
all_paths = []
#Populate Data Dict with meta data and timestamps   
for date in dates:
    for ani in os.listdir(data_path / date): 
        data_paths = list((data_path / date/ ani).rglob('*.csv'))
        if data_paths != []:
            data_dict['date'].append(data_paths[1].name.split('_')[0])
            data_dict['animal'].append(data_paths[1].name.split('_')[1])
            data_dict['task'].append(data_paths[1].name.split('_')[4])
            for ind, csv in enumerate(data_paths):
                data_dict[data_paths[ind].name.split('_')[5] +'_ts'].append(open_time(csv))
                data_dict[data_paths[ind].name.split('_')[5] +'_t0'].append(open_time(csv)[0])
df = pd.DataFrame.from_dict(data_dict)
#df = df.transpose()


#df = pd.DataFrame(data_dict)

In [ ]:
#Align to 
##df.poke1_ts = df.poke1_ts - df.top1_t0
#df.poke2_ts = df.poke2_ts - df.top1_t0
#df.spout1_ts = df.spout1_ts - df.top1_t0
#df.spout2_ts = df.spout2_ts - df.top1_t0

In [ ]:
df

In [ ]:
#Only spout on ts 
spout1_on = []
for ind, row in df.iterrows():
    x = row['spout1_ts']
    x = x[0::2]
    spout1_on.append(x)
df['spout1_on'] = spout1_on

In [ ]:
#Get all pokes_ts
pokes_ts = []
for ind, row in df.iterrows():
    x = row['poke1_ts'].astype(str).astype(float)
    y = row['poke2_ts'].astype(str).astype(float)
    z = list(itertools.chain(x,y)) 
    z = sorted(z)
    pokes_ts.append(z)
df['pokes_ts'] = pokes_ts

In [ ]:
# Number of pokes 
pokes = []
for ind, row in df.iterrows():
    x = row['pokes_ts']
    x=len(x)
    pokes.append(x)
df['poke_num'] = pokes

In [ ]:
# Time between pokes in sec
time_between_pokes = []
for ind, row in df.iterrows():
    g = row['pokes_ts']
    g = np.diff(g)
    #g = g[0::2]
    
    time_between_pokes.append(g)
df['time_between_pokes'] = time_between_pokes


In [ ]:
#Get 'trials'
## add 
trial_ts = []
for ind, row in df.iterrows():
    x = row['pokes_ts']#.astype(str).astype(float)
    y = [x[i:i+2] for i in range(0, len(x), 2)]
    z =  [x[i+1:i+3] for i in range(0, len(x), 2)]
    t = sorted(y+z)
    trial_ts.append(t)
df['trial_ts']= trial_ts


In [ ]:
#Get 'trials #'
trial_num = []
for ind, row in df.iterrows():
    x = row['trial_ts']#.astype(str).astype(float)
    y=len(x)
    trial_num.append(y)
df['trial_num']= trial_num

In [ ]:
x = r'C:\Users\nlab\Desktop\mike_bonsai\recordings\090221\G6CK3BLT\oa\090221_G6CK3BLT_control_Rig2_oa_top1DLC_resnet50_object_avoidanceAug31shuffle1_200000.h5'

row =  df.iloc[0]
num_odd_trails = len(row['poke1_ts'])
df1 = pd.DataFrame([])
dlc_positions, dlc_labels = open_h5(x)
count = -1
for c in range(num_odd_trails):
    # odd
    count += 1
    df1.at[count, 'first_poke'] = row['poke1_ts'][c]
    df1.at[count, 'second_poke'] = row['poke2_ts'][c]
    df1.at[count, 'first_spout'] = row['spout1_ts'][c]
    df1.at[count, 'second_spout'] = row['spout2_ts'][c]
    time = row['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
  
    df1.at[count, 'trail_timestamps'] = time.astype(object)
    start_stop_inds = (int(np.where([row['top1_ts']==time[0]])[1]), int(np.where([row['top1_ts']==time[-1]])[1]))
    for pos in dlc_positions:
        df1.at[count, pos] = np.array(dlc_positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
    # even
    count += 1
    if c+1 < len(row['poke1_ts']):
        df1.at[count, 'first_poke'] = row['poke2_ts'][c]
        df1.at[count, 'second_poke'] = row['poke1_ts'][c+1]
        df1.at[count, 'first_spout'] = row['spout2_ts'][c]
        df1.at[count, 'second_spout'] = row['spout1_ts'][c+1]
        time = row['top1_ts']; time = time[time > df1.loc[count,'first_poke']]; time = time[time < df1.loc[count,'second_poke']]
        df1.at[count, 'trail_timestamps'] = time.astype(object)
        start_stop_inds = (int(np.where([row['top1_ts']==time[0]])[1]), int(np.where([row['top1_ts']==time[-1]])[1]))
        for pos in dlc_positions:
            df1.at[count, pos] = np.array(dlc_positions.loc[start_stop_inds[0]:start_stop_inds[1], pos]).astype(object)
df1['animal'] = row['animal']; df1['date'] = row['date']; df1['task'] = row['task']

In [ ]:
df1

In [ ]:
plt.subplots(10,int(np.ceil(len(df1)/10)), figsize=(30,20))
count = 1
for ind, row in df1.iterrows():
    plt.subplot(10,int(np.ceil(len(df1)/10)),count)
    colors = plt.cm.jet(np.linspace(0,1,len(row['trail_timestamps'])))
    for i in range(len(row['trail_timestamps'])):
        plt.plot(row['nose_x'][i],row['nose_y'][i],'.', color=colors[i])
        plt.plot(row['obstaclewTR_x'][i],row['obstaclewTR_y'][i],'.',color='k')
        plt.plot(row['obstaclewBR_x'][i],row['obstaclewBR_y'][i],'.',color='k')
        plt.plot(row['obstaclewBL_x'][i],row['obstaclewBL_y'][i],'.',color='k')
        plt.plot(row['obstaclewTL_x'][i],row['obstaclewTL_y'][i],'.',color='k')
    count += 1
plt.plot

In [ ]:
colors = plt.cm.jet(np.linspace(0,1,len(row['trail_timestamps'])))
for i in range(len(row['trail_timestamps'])):
    plt.plot(row['nose_x'][i],row['nose_y'][i],'.', color=colors[i])
    plt.plot(row['obstaclewTR_x'][i],row['obstaclewTR_y'][i],'.',color='k')
    plt.plot(row['obstaclewBR_x'][i],row['obstaclewBR_y'][i],'.',color='k')
    plt.plot(row['obstaclewBL_x'][i],row['obstaclewBL_y'][i],'.',color='k')
    plt.plot(row['obstaclewTL_x'][i],row['obstaclewTL_y'][i],'.',color='k')